# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [28]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# Import API key
from api_keys import geoapify_key

In [29]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,alpine,32.8350,-116.7664,29.26,44,0,3.60,US,1725415353
1,1,machico,32.7000,-16.7667,22.35,69,40,7.72,PT,1725415356
2,2,wailua homesteads,22.0669,-159.3780,28.40,73,4,1.79,US,1725415358
3,3,longyearbyen,78.2186,15.6401,4.91,93,40,1.03,SJ,1725415362
4,4,luderitz,-26.6481,15.1594,15.45,64,73,4.51,NaN,1725415366


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [42]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    'Lng', 'Lat',  
    geo=True,  
    size='Humidity',  
    color='Humidity',  
    cmap='coolwarm',  
    tiles='CartoMidnight',  
    hover_cols=['City', 'Humidity'],  
    title="City Humidity Map"
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [31]:
# Narrow down cities that fit criteria and drop any results with null values
temp_criteria = (city_data_df["Max Temp"] >= 10) & (city_data_df["Max Temp"] <= 40)
humidity_criteria = city_data_df["Humidity"] < 50
filtered_cities_df = city_data_df[temp_criteria & humidity_criteria]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,alpine,32.8350,-116.7664,29.26,44,0,3.60,US,1725415353
26,26,college,64.8569,-147.8028,18.58,42,48,2.60,US,1725415417
43,43,azadshahr,34.7909,48.5707,11.27,30,0,1.03,IR,1725415439
45,45,choele choel,-39.2667,-65.6833,12.95,30,0,8.08,AR,1725415441
67,67,tshikapa,-6.4167,20.8000,25.95,37,77,0.22,CD,1725415469


### Step 3: Create a new DataFrame called `hotel_df`.

In [32]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,alpine,US,32.8350,-116.7664,44,
1,machico,PT,32.7000,-16.7667,69,
2,wailua homesteads,US,22.0669,-159.3780,73,
3,longyearbyen,SJ,78.2186,15.6401,93,
4,luderitz,NaN,-26.6481,15.1594,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",  
    "limit": 1,                           
    "apiKey": geoapify_key                
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
alpine - nearest hotel: Ayres Lodge Alpine
machico - nearest hotel: Residence Inn
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
longyearbyen - nearest hotel: The Vault
luderitz - nearest hotel: Seaview Hotel Zum Sperrgebiet
blackmans bay - nearest hotel: Villa Howden
thompson - nearest hotel: Thompson Inn
tolanaro - nearest hotel: Hôtel Mahavokey
albany - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
stanley - nearest hotel: Hotel 52
badiyah - nearest hotel: Al Areen Hotel Apartments
tadine - nearest hotel: Hôtel Nengoné
bethel - nearest hotel: Hampton Inn Danbury
invercargill - nearest hotel: The Grand
belmonte - nearest hotel: No hotel found
visaginas - nearest hotel: Gabriella
port mathurin - nearest hotel: Escale Vacances
port elizabeth - nearest hotel: Waterford Hotel
vorgashor - nearest hotel: София
dryden - nearest hotel: Holiday Inn Express
geraldton - nearest hotel: Ocean Centre Hotel
gadzhiyevo - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
0,alpine,US,32.8350,-116.7664,44,Ayres Lodge Alpine
1,machico,PT,32.7000,-16.7667,69,Residence Inn
2,wailua homesteads,US,22.0669,-159.3780,73,Hilton Garden Inn Kauai Wailua Bay
3,longyearbyen,SJ,78.2186,15.6401,93,The Vault
4,luderitz,NaN,-26.6481,15.1594,64,Seaview Hotel Zum Sperrgebiet
...,...,...,...,...,...,...
553,port saint john's,ZA,-31.6229,29.5448,64,Outback Inn
554,gua musang,MY,4.8823,101.9644,84,Hotel Usaha
555,stillwater,US,36.1156,-97.0584,70,The Atherton
556,ambato boeny,MG,-16.4667,46.7167,81,A l'Auberge Manampisoa


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:
%%capture --no-display

# Configure the map plot
city_hotel_map = hotel_df.hvplot.points(
    'Lng', 'Lat',          
    geo=True,              
    size='Humidity',       
    color='Humidity',      
    tiles='CartoMidnight',     
    cmap='coolwarm',       
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'],  
    title="City Map with Nearest Hotel Information"  
)

# Display the map
city_hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)